In [1]:
# Create a sample database model for people in an organisation
import sqlite3
import pandas as pd
con = sqlite3.connect("sample.db")
cursor = con.cursor()

In [2]:
# Tables:

cr_tabs_sql = """

DROP TABLE IF EXISTS legal_entity;

CREATE TABLE IF NOT EXISTS legal_entity (
    le_id INTEGER, 
    le_type TEXT,
    le_dob TEXT, 
    le_name TEXT);

DROP TABLE IF EXISTS staff;

CREATE TABLE IF NOT EXISTS staff (
    le_id INTEGER, 
    staff_id INTEGER, 
    staff_join_date TEXT, 
    staff_leave_date TEXT
    );

DROP TABLE IF EXISTS staff_position; 

CREATE TABLE IF NOT EXISTS staff_position (
    staff_id INTEGER, 
    role TEXT, 
    department_code TEXT
    );
    
DROP TABLE IF EXISTS department;

CREATE TABLE IF NOT EXISTS department (
    department_code TEXT, 
    org_function_code TEXT,
    description TEXT
    );

DROP TABLE IF EXISTS org_function;

CREATE TABLE IF NOT EXISTS org_function (
    org_function_code TEXT, 
    parent_org_function_code TEXT, 
    description TEXT
    )


    
"""

cursor.executescript(cr_tabs_sql)
con.commit();

In [3]:
results = cursor.execute("SELECT * from sqlite_master;")


In [4]:
pd.DataFrame(results, columns=[c[0] for c in results.description])

,type,name,tbl_name,rootpage,sql
0,table,function,function,6,CREATE TABLE function (\n function_code TEX...
1,table,legal_entity,legal_entity,2,CREATE TABLE legal_entity (\n le_id INTEGER...
2,table,staff,staff,3,"CREATE TABLE staff (\n le_id INTEGER, \n ..."
3,table,staff_position,staff_position,4,CREATE TABLE staff_position (\n staff_id IN...
4,table,department,department,5,CREATE TABLE department (\n department_code...
5,table,org_function,org_function,7,CREATE TABLE org_function (\n org_function_...


In [5]:
results = cursor.execute("SELECT * from pragma_table_info(('legal_entity'));")

In [6]:
pd.DataFrame(results, columns=[c[0] for c in results.description])

,cid,name,type,notnull,dflt_value,pk
0,0,le_id,INTEGER,0,None,0
1,1,le_type,TEXT,0,None,0
2,2,le_dob,TEXT,0,None,0
3,3,le_name,TEXT,0,None,0


In [18]:
def query_to_results_dict(conn,q):
    r_iter = conn.execute(q)
    cols = [c[0] for c in r_iter.description]
    results = []
    for r in r_iter:
        yield {c:r[e] for e,c in enumerate(cols)}

def get_sqlite_metadata(conn):
    for t in query_to_results_dict(conn,"SELECT * from sqlite_master;"):
        table_name = t['name']
        #print( t['name'])
        for c in query_to_results_dict(conn,f"SELECT * from pragma_table_info('{table_name}');"):
            #print("\t", c['name'], ":", c['type'])
            yield (t['name'], c['name'], c['type'])
        

In [19]:
q = query_to_results_dict(con,"SELECT * from sqlite_master;")
pd.DataFrame(q)

,type,name,tbl_name,rootpage,sql
0,table,function,function,6,CREATE TABLE function (\n function_code TEX...
1,table,legal_entity,legal_entity,2,CREATE TABLE legal_entity (\n le_id INTEGER...
2,table,staff,staff,3,"CREATE TABLE staff (\n le_id INTEGER, \n ..."
3,table,staff_position,staff_position,4,CREATE TABLE staff_position (\n staff_id IN...
4,table,department,department,5,CREATE TABLE department (\n department_code...
5,table,org_function,org_function,7,CREATE TABLE org_function (\n org_function_...


In [20]:
pd.DataFrame(get_sqlite_metadata(con))

,0,1,2
0,function,function_code,TEXT
1,function,parent_function_code,TEXT
2,function,description,TEXT
3,legal_entity,le_id,INTEGER
4,legal_entity,le_type,TEXT
5,legal_entity,le_dob,TEXT
6,legal_entity,le_name,TEXT
7,staff,le_id,INTEGER
8,staff,staff_id,INTEGER
9,staff,staff_join_date,TEXT


In [24]:
serial_key = { "Business Domain" : "ModelDomain", 
             "Model" : "Model", 
             "ModelType" : "ModelType", 
             "Entity" : "Class", 
             "Attribute" : "Attribute", 
             "DataType" : "DataType", 
             "Context" : "Context", 
             "SubContext" : "SubContext"}

dtype_d = { "TEXT" : "string", 
            "INTEGER" : "integer" }

In [27]:
entity_attributes=[]
for p in get_sqlite_metadata(con):
    entity_attributes.append(("Core", "Sample Database", "Physical", p[0], p[1], dtype_d.get(p[2],p[2])))

In [30]:
import csv
header = [k for k,v in serial_key.items()]
with open("sample/db_entities.csv", "w")as f:
    csvw = csv.writer(f)
    csvw.writerow(header)
    csvw.writerows(entity_attributes)